In [1]:
# 加载所需库
import numpy as np
import pandas as pd
from math import exp, pi, sqrt, ceil
from numpy.linalg import *
import networkx as nx
from scipy import sparse
from sympy import symbols, diff, solve

In [3]:
# # def data_loader(f_rpt, f_inp):


# """
# # 读取rpt文件,包含每个点的初始和最终的坐标信息
# # data_xyz 为读取到的信息
# # [[[29792], [144.0, 128.0, 88.6], [144.0, 128.0, 88.6]], ... ]
# """
# file = open(f_rpt, encoding='utf-8', mode="r")
# message = file.readlines()
# data_rpt0 = []
# for i in range(len(message)):
#     if "Field Output reported at nodes for part: PART-1-1" in message[i]:
#         i = i + 5
#         point = []
#         temp = []
#         num = 0
#         while message[i].isspace() != True:
#             s1 = message[i].split()
#             point.append([eval(s1[0])])
#             point.append([eval(s1[1])+eval(s1[4]), eval(s1[2])+eval(s1[5]), eval(s1[3])+eval(s1[6])])
#             temp.append(point.copy())
#             point.clear()
#             i = i + 1
#         data_rpt0.append(temp.copy())
#         temp.clear()
# file.close()    

# """
# # 读取inp文件,包含每条线的点信息
# # data_line 为读取到的信息
# # 输出[[29792,29793,29794,], ...]
# """
# file = open(f_inp, mode="r", encoding="utf-8")
# message = file.readlines()
# line = []
# data_inp = []
# num = 1
# for i in range(len(message)):
#     if "** PART INSTANCE: Phase2_" in message[i]:
#         j = i + 3
#         for j in range(j, len(message)):
#             if ("*Element, type=B31" in message[j]):
#                 break
#             else:
#                 s1 = message[j].strip("\n").split(",")
#                 # for k in range(len(s1)):
#                 #     if s1[k]: line.append(int(s1[k]))
#                 line.append(int(s1[0]))
#         data_inp.append(line.copy())
#         line.clear()

# file.close()

# # rpt inp 文件相互匹配
# temp = {}
# num_cnt = 0
# for i in range(len(data_inp)):
#     for j in range(len(data_inp[i])):
#         temp[data_inp[i][j]] = i+1
#         num_cnt = i+1
# for i in range(len(data_rpt0)):
#     for j in range(len(data_rpt0[i])):
#         data_rpt0[i][j][0][0] = temp[data_rpt0[i][j][0][0]]
# for i in range(len(data_rpt0)):
#     for j in range(len(data_rpt0[i])):
#         data_rpt0[i][j][1][0] = round(data_rpt0[i][j][1][0], 4)
#         data_rpt0[i][j][1][1] = round(data_rpt0[i][j][1][1], 4)
#         data_rpt0[i][j][1][2] = round(data_rpt0[i][j][1][2], 4)

In [2]:
def data_loader(f_rpt, f_inp):
    """
    # 读取rpt文件,包含每个点的初始和最终的坐标信息
    # data_xyz 为读取到的信息
    # [[[29792], [144.0, 128.0, 88.6], [144.0, 128.0, 88.6]], ... ]
    """
    file = open(f_rpt, encoding='utf-8', mode="r")
    message = file.readlines()
    data_rpt0 = []
    for i in range(len(message)):
        if "Field Output reported at nodes for part: PART-1-1" in message[i]:
            i = i + 5
            point = []
            temp = []
            num = 0
            while message[i].isspace() != True:
                s1 = message[i].split()
                point.append([eval(s1[0])])
                point.append([eval(s1[1])+eval(s1[4]), eval(s1[2])+eval(s1[5]), eval(s1[3])+eval(s1[6])])
                temp.append(point.copy())
                point.clear()
                i = i + 1
            data_rpt0.append(temp.copy())
            temp.clear()
    file.close()    

    """
    # 读取inp文件,包含每条线的点信息
    # data_line 为读取到的信息
    # 输出[[29792,29793,29794,], ...]
    """
    file = open(f_inp, mode="r", encoding="utf-8")
    message = file.readlines()
    line = []
    data_inp = []
    num = 1
    for i in range(len(message)):
        if "** PART INSTANCE: Phase2_" in message[i]:
            j = i + 3
            for j in range(j, len(message)):
                if ("*Element, type=B31" in message[j]):
                    break
                else:
                    s1 = message[j].strip("\n").split(",")
                    line.append(int(s1[0]))
            data_inp.append(line.copy())
            line.clear()

    file.close()

    # rpt inp 文件相互匹配
    temp = {}
    num_cnt = 0
    for i in range(len(data_inp)):
        for j in range(len(data_inp[i])):
            temp[data_inp[i][j]] = i+1
            num_cnt = i+1
    for i in range(len(data_rpt0)):
        for j in range(len(data_rpt0[i])):
            data_rpt0[i][j][0][0] = temp[data_rpt0[i][j][0][0]]
    for i in range(len(data_rpt0)):
        for j in range(len(data_rpt0[i])):
            data_rpt0[i][j][1][0] = round(data_rpt0[i][j][1][0], 4)
            data_rpt0[i][j][1][1] = round(data_rpt0[i][j][1][1], 4)
            data_rpt0[i][j][1][2] = round(data_rpt0[i][j][1][2], 4)

    return data_rpt0, num_cnt

In [124]:
"""
cnt为所有cnt的序列号
get_in 为所选面的输入序列号
get_out 为所选面的输出序列号
"""
def in_and_out(data_rpt, x_y_z, L, step_num, step_strain):
    """
    选择三个方向其中一个方向,找到电流流入一侧的CNT和流出一侧的CNT
    data_rpt:初始数据源
    start_or_end: "start"/"end"
    x_y_z: "xy"/"xz"/"yz"
    """
    get_in = []
    get_out = []

    if x_y_z == 'z':
        pos = 2
    elif x_y_z == 'x':
        pos = 0
    elif x_y_z == 'y':
        pos = 1
    for i in range(len(data_rpt)):
        # print(data_rpt[i][1][pos])
        if data_rpt[i][1][pos] <= 0.1: get_in.append(data_rpt[i][0][0])
        # if data_rpt[i][1][pos] >= L-0.1: get_out.append(data_rpt[i][0][0])
        if data_rpt[i][1][pos] >= (L + (step_num * step_strain) - 0.1): get_out.append(data_rpt[i][0][0])
        
    get_in = list(set(get_in))
    get_out = list(set(get_out))
    return get_in, get_out

# cnt = list(set([i[0][0] for i in data_rpt0[1]]))
# get_in = in_and_out(data_rpt0[1], xyz, 100)[0]
# get_out = in_and_out(data_rpt0[1], xyz, 100)[1]
# print(len(get_in), len(get_out))
# print(get_in)
# print(get_out)

In [127]:
"""
找出栅格中存在相邻点的cnt序列号
L
d_min
"""
def grid_method(data_rpt, L, d_min, step_num, step_strain_x, step_strain_yz):
    """
    point:是包含二维坐标信息的值
    L:是RVE边长
    d_min:是栅格距离
    return 返回栅格中大于等于两个点的相邻点信息 [[],[]...[]]
    """
    L_cur_x = L + (step_num * step_strain_x) - 0.05
    L_cur_yz = L + (step_num * step_strain_yz) - 0.05
    grid_num_x = ceil(L_cur_x / d_min)
    grid_num_yz = ceil(L_cur_yz / d_min)
    grid = [[] for i in range(grid_num_x*grid_num_yz*grid_num_yz)]

    for i in range(len(data_rpt)):
        if (data_rpt[i][1][0] < 0): data_rpt[i][1][0] = 0
        if (data_rpt[i][1][1] < 0): data_rpt[i][1][1] = 0
        if (data_rpt[i][1][2] < 0): data_rpt[i][1][2] = 0
        if (data_rpt[i][1][0] >= L_cur_x): data_rpt[i][1][0] = L_cur_x - 0.01
        if (data_rpt[i][1][1] >= L_cur_yz): data_rpt[i][1][1] = L_cur_yz - 0.01
        if (data_rpt[i][1][2] >= L_cur_yz): data_rpt[i][1][2] = L_cur_yz - 0.01

        num = (data_rpt[i][1][0] // d_min) + (data_rpt[i][1][1] // d_min) * grid_num_yz + (data_rpt[i][1][2] // d_min) * grid_num_yz**2
        grid[int(num)].append(data_rpt[i][0][0])

    ans = []
    for i in range(len(grid)):
        grid[i] = list(set(grid[i]))
        for j in range(len(grid[i])):
            for k in range(j+1, len(grid[i])):
                if [grid[i][j], grid[i][k]] in ans: continue
                else: 
                    d = distance(data_rpt, grid[i][j], grid[i][k])
                    if d < 0.54:
                        ans.append([grid[i][j], grid[i][k]])
    return ans

# cnt_part_link = grid_method(data_rpt[0], L, d_min)
# print(cnt_part_link)

In [6]:
def dis(A, B, C, D):
    s1, s2 = symbols('s1, s2')
    P = A + s1 * (B - A)
    Q = C + s2 * (D - C)
    op = np.sum(np.square(P-Q))
    equation1 = diff(op, s1)
    equation2 = diff(op, s2)
    ans = solve([equation1, equation2],[s1, s2])
    s1 = float(ans[s1])
    s2 = float(ans[s2])

    if s1 > 1: s1 = 1
    elif s1 < 0: s1 = 0

    if s2 > 1: s2 = 1
    elif s2 < 0: s2 = 0

    P1 = A + s1 * (B - A)
    Q1 = C + s2 * (D - C)
    dis = np.sqrt(float(np.sum(np.square(P1-Q1))))
    return dis

In [7]:
def distance(data_rpt, cnt1, cnt2):
    xyz1 = []
    xyz2 = []

    for i in range(len(data_rpt)):
        if data_rpt[i][0][0] == cnt1: xyz1.append(data_rpt[i][1])
        if data_rpt[i][0][0] == cnt2: xyz2.append(data_rpt[i][1])
    
    d = 150
    # a = 0
    # b = 0
    # for i in range(len(xyz1)-1):
    #     A = np.array(xyz1[i])
    #     B = np.array(xyz1[i+1])
    #     for j in range(len(xyz2)-1):
    #         C = np.array(xyz2[j])
    #         D = np.array(xyz2[j+1])
    #         op1=np.linalg.norm(A-C)
    #         op2=np.linalg.norm(A-D)
    #         op3=np.linalg.norm(B-C)
    #         op4=np.linalg.norm(B-D)
    #         d_op = min(op1, op2, op3, op4)
    #         if d_op < d:
    #             d = d_op
    #             a = i 
    #             b = j
    # d = dis(np.array(xyz1[a]),np.array(xyz1[a+1]),np.array(xyz2[b]),np.array(xyz2[b+1]))
    for x1, y1, z1 in xyz1:
        for x2, y2, z2 in xyz2:
            d = min(d, sqrt((x2-x1)**2+(y2-y1)**2+(z2-z1)**2)*0.5)
    return d

In [8]:
def concat(cnt_part_link): 
    # l = [[1,2],[3,4,5],[0,4]]
    G = nx.Graph()
    #将节点添加到Graph    
    G.add_nodes_from(sum(cnt_part_link, []))
    #从节点列表创建边
    q = [[(s[i],s[i+1]) for i in range(len(s)-1)] for s in cnt_part_link]
    for i in q:
        #向Graph添加边
        G.add_edges_from(i)
    #查找每个组件的图形和列表节点中的所有连接组件
    cnt_path = [list(i) for i in nx.connected_components(G)]
    return cnt_path

In [9]:
"""
找到联通通路中真实存在于输入输出的cnt
"""
def real_in_out(cnt_path, get_in, get_out):
    real_get_in = []
    real_get_out = []
    for i in cnt_path:
        a = [val for val in get_in if val in i]
        b = [val for val in get_out if val in i]
        real_get_in.append(a)
        real_get_out.append(b)
  
    a = [i for j in real_get_in for i in j]
    b = [i for j in real_get_out for i in j]
    return a,b

# real_get_in = real_in_out(cnt_path, get_in, get_out)[0]
# real_get_out = real_in_out(cnt_path, get_in, get_out)[1]
# print(real_get_in)
# print(real_get_out)

In [10]:
def get_line(real_get_in, real_get_out, input, cnt_path, cnt_part_link, ans, first):
    if input == []: return
    
    first = first + input
    mid_ans = []
    for cnt in input:
        if cnt in real_get_out: continue

        cnt_neighbor = [i for i in cnt_part_link if cnt in i]
        cnt_neighbor = list(set(sum(cnt_neighbor,[])))
        cnt_neighbor.remove(cnt)
        cnt_neighbor = list(set(cnt_neighbor) - set(first))
        for i in cnt_neighbor:
            ans.append([cnt, i])
        mid_ans.append(cnt_neighbor)
    
    mid_ans = list(set([i for j in mid_ans for i in j]))
    tmp = [j for j in mid_ans if j in first]
    mid_ans = list(set(mid_ans) - set(tmp))
    # print(len(ans))
    get_line(real_get_in, real_get_out, mid_ans, cnt_path, cnt_part_link, ans, first)
    return ans

In [11]:
# 计算电阻值
# 注意A中的D
def R_ans(d):
    D = 50*10**(-9)
    d = (d*100*10**(-9)) - D
    
    h = 6.62606957*(10**(-34))
    A = pi*D*D/4
    e = 1.602176565*(10**(-19))
    m = 9.10938291*(10**(-31))
    ev = 1.6*(10**(-19))   
    lamut = 1*ev
    
    try:
        R = (h*h*d)/(A*e*e*sqrt(2*m*lamut))*exp(4*pi*d/h*sqrt(2*m*lamut))
    except OverflowError:
        R = float('inf')
    # print(R)
    return R

In [120]:
def ans(num_cnt, line, real_get_in, real_get_out, cnt_path, cnt, data_rpt):
    """
    
    """
    c = [i for j in cnt_path for i in j]
    invalid = [val for val in cnt if val not in c]
    # 
    A = np.zeros((int(num_cnt), int(len(line))))
    for i in range(len(line)):
        if line[i][0] in real_get_in:
            A[real_get_in[0]-1][i] = 1
            A[line[i][1]-1][i] = -1
        elif line[i][1] in real_get_out:
            A[line[i][0]-1][i] = 1
            A[real_get_out[0]-1][i] = -1
        else:
            A[line[i][0]-1][i] = 1
            A[line[i][1]-1][i] = -1
            
    mm = A[real_get_in[0]-1].copy()

    delet = []
    for i in range(len(real_get_in)):
        delet.append(real_get_in[i]-1)
    for i in range(len(real_get_out)):
        delet.append(real_get_out[i]-1)
    for i in range(len(invalid)):
        delet.append(invalid[i]-1)

    A = np.delete(A, delet, axis=0)
    A = np.vstack((mm, A))

    delet = []
    for i in range(len(A)):
        if np.any(A[i]) == False:
            delet.append(i)
    A = np.delete(A, delet, axis=0)

    # 
    Gb = []
    for i in range(len(line)):
        D = distance(data_rpt, line[i][0],line[i][1])
        if D < 0.5034: Gb.append(1/R_ans(0.5034))
        else: Gb.append(1/R_ans(D))
    
    sA = sparse.csr_matrix(A)
    Gb = np.array(Gb)
    offset = np.array([0])
    sGb = sparse.dia_matrix((Gb, offset),shape=(Gb.shape[0], Gb.shape[0])).tocsr()
    Gn = (sA*sGb)*(sA.T).A
    # 
    I = np.zeros((len(Gn), 1))
    I[0] = 1

    tmp = np.linalg.solve(Gn, I)[0][0]
    ans = 1/(tmp*10**(-5))
    return (tmp, ans)

In [130]:
"""
固定所需要的参数
xyz方向
文件路径
"""
path = "../CNT_M_0D011/1/"
name = "Job_CNT_M_0D011"
f_rpt = path + "abaqus.rpt"
f_inp = path + name + ".inp"
# xyz = "x"
L = 100
d_min = 2
step_strain_x = 0.2
step_strain_yz = -0.0005

In [131]:
data_rpt0, num_cnt = data_loader(f_rpt, f_inp)

In [132]:
data_1 = []
data_2 = []
data_3 = []
data_4 = []
data_5 = []
data_6 = []
for i in ['x','y','z']:
    xyz = i
    data1 = []
    data2 = []
    data_rpt_copy = data_rpt0.copy()
    step_num = 0
    for data_rpt in data_rpt_copy:
        print(step_num)
        if xyz == 'x':
            cnt = list(set([i[0][0] for i in data_rpt]))
            get_in, get_out = in_and_out(data_rpt, xyz, L, step_num, step_strain_x)
            # print(len(get_in), len(get_out))
        else:
            cnt = list(set([i[0][0] for i in data_rpt]))
            get_in, get_out = in_and_out(data_rpt, xyz, L, step_num, step_strain_yz)

        if get_in == [] or get_out == []: 
            data1.append(0)
            data2.append(0)
            step_num += 1
            print(0,0)
            continue
        else:
            cnt_part_link = grid_method(data_rpt, L, d_min, step_num, step_strain_x, step_strain_yz)
            cnt_path = concat(cnt_part_link)
            real_get_in, real_get_out = real_in_out(cnt_path, get_in, get_out)
            if real_get_in == [] or real_get_out == []: 
                data1.append(0)
                data2.append(0)
                step_num += 1
                print(0,0)
                continue
            else:
                line = line = get_line(real_get_in, real_get_out, real_get_in, cnt_path, cnt_part_link, [], [])
                try:
                    R, det_R = ans(num_cnt, line, real_get_in, real_get_out, cnt_path, cnt, data_rpt)
                    data1.append(R)
                    data2.append(det_R)
                    step_num += 1
                    print(R, det_R)
                except:
                    data1.append(0)
                    data2.append(0)
                    step_num += 1
                    print(0,0)
    if xyz == 'x':
        data_1 = data1
        data_2 = data2
    if xyz == 'y':
        data_3 = data1
        data_4 = data2
    if xyz == 'z':
        data_5 = data1
        data_6 = data2

In [122]:
data = pd.DataFrame({'R_11':data_1, 'S/m_11':data_2, 'R_22':data_3, 'S/m_22':data_4, 'R_33':data_5, 'S/m_33':data_6})
data.to_csv(path + name + ".csv", index=False)

In [45]:
data = pd.DataFrame({'R_11':data_1, 'S/m_11':data_2})
data.to_csv(r'F:\Data\CNT_M_0D001\1\CNT_M_0D001_data.csv', index=False)

In [49]:
data['R_22'] = data_3
data['S/m_22'] = data_4
data['R_33'] = data_5
data['S/m_33'] = data_6
data.to_csv(r"F:\Data\CNT_M_0D001\1\CNT_M_0D001_data.csv")
# a1 = pd.DataFrame({'R_22':data_1, 'S/m_22':data_2})
# a1 = data.join(a1)
# a1.to_csv(r'F:\Data\CNT_M_0D001\1\CNT_M_0D001_data.csv', index=False)

ValueError: Length of values (101) does not match length of index (91)

In [29]:
a2 = pd.DataFrame({'R_33':data_1, 'S/m_33':data_2})
a2 = data.join(a2)
a2.to_csv(r'F:\Data\CNT_M_0D001\1\CNT_M_0D001_data.csv',mode='w', index=False)